In [1]:
# -*- coding:utf-8 -*-

### 5.3.1  读取ElasticSearch数据

In [2]:
from elasticsearch import Elasticsearch

es_host = '127.0.0.1' # 宿主机IP地址
es_index = 'test_1'
es = Elasticsearch(es_host) # 建立ES连接
result = es.search(index=es_index, body={}, size=10) # 查询并返回前10条数据
for item in result['hits']['hits']: # 访问返回数据
    print(item)

{'_index': 'test_1', '_type': 'product', '_id': 'hx7gl2sBBskLuq6uThLI', '_score': 1.0, '_source': {'author': 'Xie Yan'}}


### 5.3.2  读取S3云存储数据

In [3]:
import boto 
import boto.s3.connection 
from boto.s3.key import Key 

s3_aws_access_key_id = 'accessKey1'
s3_aws_secret_access_key = 'verySecretKey1'
s3_host = '127.0.0.1'
s3_port = 8000

conn = boto.connect_s3(aws_access_key_id = s3_aws_access_key_id, 
                       aws_secret_access_key = s3_aws_secret_access_key, 
                       host = s3_host, 
                       port=s3_port, 
                       is_secure = False, 
                       calling_format = boto.s3.connection.OrdinaryCallingFormat())

In [4]:
# 列出所有存储桶
rs = conn.get_all_buckets() 
for i in rs:
    print(i)

model_bucket = 'tmp'
model_bucket_exist = conn.lookup(model_bucket) # 查找桶是否存在
if model_bucket_exist:
    print("exist", model_bucket_exist)  
    mybucket = conn.get_bucket(model_bucket) # 获取桶句柄
else:
    print("not exist")
    mybucket = conn.create_bucket('tmp') # 创建一个新桶

not exist


In [5]:
# 上传文件
k = Key(mybucket)
k.key = 'test_file'
filename = 'testfile.txt'
k.set_contents_from_filename(filename) 
# 列出桶中文件
mybucket = conn.get_bucket(model_bucket)
print(mybucket.get_all_keys(maxkeys=5)) 
# 下载文件
filename2 = 'testfile2.txt'
k.get_contents_to_filename(filename2)

[<Key: tmp,test_file>]


In [6]:
mybucket.delete_key('test_file') # 删除文件
conn.delete_bucket(model_bucket) # 删除桶
conn.close()

### 5.3.3  读取Hive数据

In [7]:
from impala.dbapi import connect
# port为docker中10000向外映射的端口号
conn = connect(host="localhost", port=32777,
     database="default", auth_mechanism="PLAIN")
cur = conn.cursor()
sql = "create table if not exists test_table(id int)"   # SQL语句：创建数据表
cur.execute(sql)
sql = "show tables"  # 显示所有数据表名
cur.execute(sql)
print(cur.fetchall())
sql = "select * from default.test_table"  # 查看数据表内容
cur.execute(sql)
print(cur.fetchall())
conn.close()

[('test_table',)]
[]
